# QLoRA fine‑tuning for distilbert-base-uncased

Upload `dataset.csv` with `text,label` columns and set `DATASET_PATH` as needed.

In [ ]:

# If on Colab:
# !pip install -q transformers datasets peft accelerate evaluate scikit-learn matplotlib bitsandbytes --upgrade

import os, numpy as np, pandas as pd, evaluate, matplotlib.pyplot as plt
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, TrainingArguments, Trainer)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import numpy as np

DATASET_PATH = "/content/dataset.csv"  # CSV with columns: text,label
MODEL_NAME = "distilbert-base-uncased"
USE_QLORA = True  # True -> 4bit QLoRA via bitsandbytes
OUTPUT_DIR = "outputs_distilbert_qlora_adv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df = pd.read_csv(DATASET_PATH)
assert "text" in df.columns and "label" in df.columns
labels = sorted(df["label"].astype(str).unique().tolist())
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}
df["label_id"] = df["label"].astype(str).map(label2id)

# Split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label_id"])
train_df, val_df  = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df["label_id"])

# HF Dataset
train_ds = Dataset.from_pandas(train_df[["text","label_id"]])
val_ds   = Dataset.from_pandas(val_df[["text","label_id"]])
test_ds  = Dataset.from_pandas(test_df[["text","label_id"]])

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess(ex):
    out = tokenizer(ex["text"], truncation=True, max_length=256)
    out["labels"] = ex["label_id"]
    return out

train_ds = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)
test_ds  = test_ds.map(preprocess, batched=True, remove_columns=test_ds.column_names)

collator = DataCollatorWithPadding(tokenizer=tokenizer)

bnb_config = None
if USE_QLORA:
    from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype="bfloat16",
    )
    base_model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=len(labels), id2label=id2label, label2id=label2id,
        quantization_config=bnb_config, device_map="auto"
    )
    base_model = prepare_model_for_kbit_training(base_model, use_gradient_checkpointing=True)
else:
    base_model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=len(labels), id2label=id2label, label2id=label2id
    )

lora_cfg = LoraConfig(
    r=16, lora_alpha=32,
    target_modules=["query","key","value","dense","fc1","fc2","classifier","intermediate.dense","output.dense"],
    lora_dropout=0.05, bias="none", task_type=TaskType.SEQ_CLS
)
model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds = np.argmax(logits, axis=-1)
    f1_w = f1_score(labels_np, preds, average="weighted")
    f1_m = f1_score(labels_np, preds, average="macro")
    acc = accuracy_score(labels_np, preds)
    return {"f1_weighted": f1_w, "f1_macro": f1_m, "accuracy": acc}

args = TrainingArguments(
    OUTPUT_DIR, per_device_train_batch_size=16, per_device_eval_batch_size=32,
    learning_rate=2e-4, num_train_epochs=4, weight_decay=0.05,
    evaluation_strategy="epoch", save_strategy="epoch", load_best_model_at_end=True,
    logging_steps=50, fp16=True, gradient_accumulation_steps=2,
    save_total_limit=2, report_to="none"
)

trainer = Trainer(
    model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds,
    tokenizer=tokenizer, data_collator=collator, compute_metrics=compute_metrics
)

trainer.train()
eval_res = trainer.evaluate(test_ds)
print("Test metrics:", eval_res)

# Confusion matrix
preds = np.argmax(trainer.predict(test_ds).predictions, axis=-1)
cm = confusion_matrix(test_ds["labels"], preds, labels=list(range(len(labels))))
plt.figure(); plt.imshow(cm, interpolation='nearest'); plt.title('Confusion Matrix'); plt.colorbar()
plt.tight_layout(); plt.ylabel('True'); plt.xlabel('Pred')
cm_path = os.path.join(OUTPUT_DIR, "confusion_matrix.png"); plt.savefig(cm_path, dpi=150)

# Save adapter
trainer.model.save_pretrained(os.path.join(OUTPUT_DIR, "lora_adapter"))
tokenizer.save_pretrained(OUTPUT_DIR)

# Save final metrics
with open(os.path.join(OUTPUT_DIR, "report.json"), "w") as f:
    import json; json.dump(eval_res, f, indent=2)
print("Saved report, confusion matrix, and LoRA adapter to", OUTPUT_DIR)
